# Predicting Browsing Selection of Ungulates

This notebook contains a pipeline which produces estimates of ungulate browsing selection in a forested area.

The pipeline consists of the following steps:

1. [Loading preliminaries](#1)
2. [Loading the dataset](#2)
3. [Generating estimates](#3)
4. [Displaying results](#4)

Each of the steps are described in more detail below. To use the notebook, please save the woody plant supply data on the forested area in question to the same folder as this notebook and in the format described in the second section. Afterwards, please hit *Run all cells* and, once the notebook has finished running, find the output of the prediction in the last section of the notebook as well as in a file saved in the same directory as this notebook under the name *browsing_forecast.csv*.

## 1. Loading preliminaries <a class="anchor" id="1"></a>

The cell below loads packages which are required for the computations in this notebook.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import GradientBoostingRegressor

import joblib

from matplotlib import pyplot as plt

from IPython.display import clear_output

import warnings

from typing import Dict, Union

import math

The next cell loads some auxilliary functionality which is utilized later in this notebook.

In [2]:
def normalize_TS(X:np.ndarray) -> np.ndarray:
    '''Normalizes the TS columns of a predictor array.
    :param: X: the array which contains the columns to normalize.
    :return: the complete array in which the TS columns are now normalized.'''

    X_normalized = X.copy()

    cols_to_normalize = [i for i in range(np.shape(X)[1]) if i % 3 == 0]

    smoothing = 0.00001     # Applied so that 0 division does not cause any problems while normalizing.

    X_normalized[:, cols_to_normalize] = (X[:, cols_to_normalize] - (X[:, cols_to_normalize].min(axis=0))) \
                            / (X[:, cols_to_normalize].max(axis=0) - (X[:, cols_to_normalize].min(axis=0)) + smoothing)
    
    return X_normalized

def adjust_predictions(y_pred:np.ndarray, X:np.ndarray, noSupply_value:float) -> np.ndarray:
    '''Ensures that the specified noSupply_value is predicted whenever there is no shoot supply
    and that there are no negative predictions when there IS supply.
    :param: y_pred: the array of original predictions.
    :param: X: the array of predictors.
    :param: noSupply_value: the value to predict in case of no supply.
    :return: the adjusted array of predictions.
    '''

    y_pred_adjusted = y_pred.copy()

    for i in range(y_pred.shape[0]):
            for j in range(y_pred.shape[1]):
                if X[i, j * 3] == 0:
                    y_pred_adjusted[i, j] = noSupply_value

                elif y_pred[i, j] < 0:
                    y_pred_adjusted[i, j] = 0

    return y_pred_adjusted

## 2. Loading the dataset <a class="anchor" id="2"></a>

This pipeline allows you to generate browsing estimates based on the available plant supply in a forested area. For this purpose, you need to have data saved on the plant supply in the same folder as this notebook. The data should be in an Excel file named `forest_data.xlsx`, and it should be in the following format:

* The worksheet containing the data should be the only sheet in the Excel file.

* The table in the worksheet should contain exactly 27 columns.

* The table can have an arbitrary number of rows, each of which represents a point within the forest in question. The rows should be ordered so that the points follow each other as they would along a transect in the forest (i.e., adjacent points in the forest should be in adjacent rows).

* Please make sure that the number of rows (i.e., the number of sampling points) is a multiple of 5. This is because the sampling points will be aggregated into sections of the forest, each of which should comprise 5 sampling points, in order to help the performance of the predictive models.

* Each of the 27 columns should contain the total number of shoots found of a given species at the sampling point contained in the specific row. The columns should correspond to the following species **in the exact same order**:

    1. Unknown species
    2. Quercus petraea
    3. Quercus cerris
    4. Fraxinus excelsior
    5. Fraxinus ornus
    6. Carpinus betulus
    7. Fagus sylvatica
    8. Acer pseudoplatanus
    9. Acer platanoides
    10. Acer campestre
    11. Pinus sylvestris
    12. Robinia pseudoacacia
    13. Ligustrum vulgare
    14. Crataegus monogyna
    15. Cornus mas
    16. Cornus sanguinea
    17. Prunus spinosa
    18. Rubus fruticosus
    19. Rosa canina
    20. Acer tataricum
    21. Prunus avium
    22. Corylus avellana
    23. Ulmus minor 
    24. Sorbus aucuparia
    25. Pyrus pyraster
    26. Euonymus verrucosus
    27. Quercus pubescens
    
    <br>
* The first row of the table should contain the names of the species as headers. The numeric values should start in the **second row** of the Excel sheet.

You can find an example of such a file contained in the folder named *Example*. You can use it as a template to create your own file containing woody plant supply data.

The following cells use this file to load the dataset and calculate frequency (F) and relative proportion of supply (RPS) values for each of the species. Afterwards, the sampling points are aggregated by fives so that these aggregated instances represent sections of the forest instead of points, which, as mentioned above, helps the performance of the machine learning models. The processed dataset will be saved in the variable named `data`.

In [3]:
# Loading the data and saving total shoots (TS) values as well as all species names.

loaded_dataset = pd.read_excel('forest_data.xlsx')

ts_values = loaded_dataset.values

species = loaded_dataset.columns.values

In [4]:
# Calculating distribution properties frequency (F) and relative proportion of supply (RPS) for each species.

# A table is created, where for each species these properties are stored.
# The first row contains the species names, the second the F and the third the RPS values.

sampling_points = len(ts_values)
all_shoots = 0

nr_species = len(species)

properties_table = np.concatenate((species[np.newaxis, :].astype(object), np.zeros((2, nr_species))))

for i in range(1,len(ts_values)):
    for j in range(nr_species):
        ts = ts_values[i, j]
            
        if ts != 0:
            properties_table[1,j] += 1
            properties_table[2,j] += ts

            all_shoots += ts
    
#Normalizing species data with nr of sampling points / shoots in the forest.
properties_table[1, :] = properties_table[1, :] / sampling_points
properties_table[2, :] = properties_table[2, :] / all_shoots

In [5]:
# Creating a new dataset table where F and RPS values are included along with TS values.

unaggregated_data = np.zeros((ts_values.shape[0], nr_species * 3))

f_values = properties_table[1, :][np.newaxis, :]
rps_values = properties_table[2, :][np.newaxis, :]

for i in range(nr_species):
    f_value = f_values[0, i]
    f_array = np.tile(f_value, (np.shape(ts_values)[0])).astype(object)

    rps_value = rps_values[0, i]
    rps_array = np.tile(rps_value, (np.shape(ts_values)[0])).astype(object)

    unaggregated_data[:, i * 3] = ts_values[:, i]
    unaggregated_data[:, i * 3 + 1] = f_array
    unaggregated_data[:, i * 3 + 2] = rps_array

unaggregated_data = unaggregated_data.astype(float)

In [6]:
# Aggregating the dataset to represent sections of the forest comprising 5 adjacent sampling points each.

data = np.zeros((unaggregated_data.shape[0] // 5, nr_species * 3))

for i in range(len(data)):
    for j in range(nr_species):
        start_index = i * 5
        end_index = start_index + 4
        
        data[i, j*3] = np.sum(unaggregated_data[start_index:end_index+1, j*3].astype(float))
        data[i, j*3+1] = unaggregated_data[start_index, j*3+1]
        data[i, j*3+2] = unaggregated_data[start_index, j*3+2]

The following cell perform normalization on the TS values so that they are projected on the interval [0,1].

In [7]:
data = normalize_TS(data)

## 3. Generating estimates <a class="anchor" id="3"></a>

The following cell loads the machine learning models, namely Gradient Boosting Regressors, each fitted to predict the extent of browsing on a different species within the forest.

In [8]:
models = []

for i in range(nr_species):
    models.append(joblib.load(f'./Models/gb_{i}.pkl'))

The following cell generates estimates of browsing extent for each of the species in each of the forest sections represented in the dataset.

In [9]:
# The value to predict if there was no supply from a given species within a section of the forest
# (and consequently no possibility of browsing):
noSupply_value = -0.00001

section_estimates = np.zeros((len(data), nr_species))

for i in range(nr_species):
    predicted_browsing = models[i].predict(data)
    
    section_estimates[:, i] = models[i].predict(data)

section_estimates = adjust_predictions(section_estimates, data, noSupply_value)

The following cell calculates the overall estimated browsing extent on each of the species within the forest.

This is done by summing the estimated browsing extents in each of the sections weighted by the contribution of the section to the overall number of shoots offered by a given species within the whole forest.

In [10]:
forest_estimates = np.zeros((1, nr_species))

for sp in range(nr_species):
    total_shoots = np.sum(data[:, sp*3])

    if total_shoots != 0:

        section_contributions = [data[section, sp*3] / total_shoots for section in range(len(data))]

        weighted_estimates = [contribution * section_estimates[index, sp] for index, contribution in enumerate(section_contributions)]

        forest_estimates[0, sp] = sum(weighted_estimates)

    else:
        forest_estimates[0, sp] = noSupply_value

## 4. Displaying results <a class="anchor" id="4"></a>

The results of the forecast can be found below. The first cell prints them, the second saves them in a .csv file in the same folder as this notebook, named `browsing_forecast.csv`.

In [11]:
print(f'{"SPECIES":<20}\t{"TOTAL SHOOTS":>12}\t{"FORECAST":>8}')

for index, sp in enumerate(species.tolist()):
    forecast = forest_estimates[0, index]

    total_shoots = np.sum(ts_values[:, index])

    if forecast < 0:
        print(f'{sp:<20}\t{total_shoots:>12.0f}\t{"NaN":>8}')

    else:
        print(f'{sp:<20}\t{total_shoots:>12.0f}\t{f"{forecast*100:.1f}%":>8}')

SPECIES             	TOTAL SHOOTS	FORECAST
Unknown             	           6	   66.7%
Quercus petraea     	         165	    7.8%
Quercus cerris      	          70	    0.0%
Fraxinus excelsior  	           0	     NaN
Fraxinus ornus      	         543	    7.3%
Carpinus betulus    	           3	    8.3%
Fagus sylvatica     	           0	     NaN
Acer pseudoplatanus 	          38	    0.0%
Acer platanoides    	           0	     NaN
Acer campestre      	          84	    2.3%
Pinus sylvestris    	           0	     NaN
Robinia pseudoacacia	          13	    5.8%
Ligustrum vulgare   	         473	   30.9%
Crataegus monogyna  	         624	   11.5%
Cornus mas          	         135	   14.5%
Cornus sanguinea    	           0	     NaN
Prunus spinosa      	         341	    4.6%
Rubus fruticosus    	          59	    2.6%
Rosa canina         	         333	    7.7%
Acer tataricum      	          58	   11.9%
Prunus avium        	           0	     NaN
Corylus avellana    	           0	     NaN
Ulmus minor

In [12]:
output = np.zeros((nr_species, 3)).astype('object')

output[:, 0] = species

for index, sp in enumerate(species.tolist()):
    forecast = forest_estimates[0, index]

    total_shoots = np.sum(ts_values[:, index])

    output[index, 1] = total_shoots

    output[index, 2] = forecast

output = np.vstack((['Species', 'Total shoots', 'Forecast'], output))

np.savetxt('browsing_forecast.csv', output, delimiter=',', fmt='%s')